### Spark session configuration
This cell sets Spark session settings to enable _Verti-Parquet_ and _Optimize on Write_. More details about _Verti-Parquet_ and _Optimize on Write_ in tutorial document.

In [5]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 7, Finished, Available, Finished)

#### Approach #1 - sale_by_date_city
In this cell, you are creating three different Spark dataframes, each referencing an existing delta table.

In [9]:
df = spark.read.format("delta").load("Tables/fact_sale")
df.write.format("delta").saveAsTable("dbo.fact_sale")


StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 12, Finished, Available, Finished)

In [13]:
# Load from existing Delta folder
df = spark.read.format("delta").load("Tables/dimension_city")

# Save under dbo schema
df.write.format("delta").saveAsTable("dbo.dimension_city")


StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 16, Finished, Available, Finished)

In [14]:
# List all tables
spark.sql("SHOW TABLES").show()

# Check only dbo schema
spark.sql("SHOW TABLES IN dbo").show()


StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 17, Finished, Available, Finished)

+--------------------+------------------+-----------+
|           namespace|         tableName|isTemporary|
+--------------------+------------------+-----------+
|`Project WS1`.wwi...|    dimension_city|      false|
|`Project WS1`.wwi...|dimension_customer|      false|
|`Project WS1`.wwi...|    dimension_date|      false|
|`Project WS1`.wwi...|         fact_sale|      false|
+--------------------+------------------+-----------+

+--------------------+------------------+-----------+
|           namespace|         tableName|isTemporary|
+--------------------+------------------+-----------+
|`Project WS1`.wwi...|    dimension_city|      false|
|`Project WS1`.wwi...|dimension_customer|      false|
|`Project WS1`.wwi...|    dimension_date|      false|
|`Project WS1`.wwi...|         fact_sale|      false|
+--------------------+------------------+-----------+



In [15]:
df_fact_sale = spark.read.table("dbo.fact_sale")
df_dimension_date = spark.read.table("dbo.dimension_date")
df_dimension_city = spark.read.table("dbo.dimension_city")


StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 18, Finished, Available, Finished)

In this cell, you are joining these tables using the dataframes created earlier, doing group by to generate aggregation, renaming few of the columns and finally writing it as delta table in the _Tables_ section of the lakehouse.

In [16]:
sale_by_date_city = df_fact_sale.alias("sale") \
.join(df_dimension_date.alias("date"), df_fact_sale.InvoiceDateKey == df_dimension_date.Date, "inner") \
.join(df_dimension_city.alias("city"), df_fact_sale.CityKey == df_dimension_city.CityKey, "inner") \
.select("date.Date", "date.CalendarMonthLabel", "date.Day", "date.ShortMonth", "date.CalendarYear", "city.City", "city.StateProvince", "city.SalesTerritory", "sale.TotalExcludingTax", "sale.TaxAmount", "sale.TotalIncludingTax", "sale.Profit")\
.groupBy("date.Date", "date.CalendarMonthLabel", "date.Day", "date.ShortMonth", "date.CalendarYear", "city.City", "city.StateProvince", "city.SalesTerritory")\
.sum("sale.TotalExcludingTax", "sale.TaxAmount", "sale.TotalIncludingTax", "sale.Profit")\
.withColumnRenamed("sum(TotalExcludingTax)", "SumOfTotalExcludingTax")\
.withColumnRenamed("sum(TaxAmount)", "SumOfTaxAmount")\
.withColumnRenamed("sum(TotalIncludingTax)", "SumOfTotalIncludingTax")\
.withColumnRenamed("sum(Profit)", "SumOfProfit")\
.orderBy("date.Date", "city.StateProvince", "city.City")

sale_by_date_city.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/aggregate_sale_by_date_city")

StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 19, Finished, Available, Finished)

#### Approach #2 - sale_by_date_employee
In this cell, you are creating a temporary Spark view by joining 3 tables, doing group by to generate aggregation, renaming few of the columns. 

In [22]:
df = spark.read.format("delta").load("Tables/dimension_employee")


StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 25, Finished, Available, Finished)

In [23]:
df.write.format("delta").mode("overwrite").saveAsTable("dbo.dimension_employee")


StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 26, Finished, Available, Finished)

In [26]:
df_joined = spark.sql("""
    SELECT
        DD.Date, 
        DD.CalendarMonthLabel,
        DD.Day, 
        DD.ShortMonth AS Month, 
        DD.CalendarYear AS Year,
        DE.PreferredName, 
        DE.Employee,
        SUM(FS.TotalExcludingTax) AS SumOfTotalExcludingTax,
        SUM(FS.TaxAmount) AS SumOfTaxAmount,
        SUM(FS.TotalIncludingTax) AS SumOfTotalIncludingTax,
        SUM(FS.Profit) AS SumOfProfit
    FROM dbo.fact_sale FS
    INNER JOIN dbo.dimension_date DD ON FS.InvoiceDateKey = DD.Date
    INNER JOIN dbo.dimension_employee DE ON FS.SalespersonKey = DE.EmployeeKey
    GROUP BY 
        DD.Date, DD.CalendarMonthLabel, DD.Day, DD.ShortMonth, DD.CalendarYear,
        DE.PreferredName, DE.Employee
""")


StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 29, Finished, Available, Finished)

In [27]:
df_joined.write.format("delta").mode("overwrite").saveAsTable("dbo.sale_by_date_employee")


StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 30, Finished, Available, Finished)

In this cell, you are reading from the temporary Spark view created in the previous cell and and finally writing it as delta table in the _Tables_ section of the lakehouse.

In [28]:
sale_by_date_employee = spark.sql("SELECT * FROM sale_by_date_employee")
sale_by_date_employee.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/aggregate_sale_by_date_employee")

StatementMeta(, 044e2740-b0a1-4151-b53f-0939714f9a46, 31, Finished, Available, Finished)